<!--
# Cineromycin B
### Bayesian inference of conformational state populations from computational models and sparse experimental observables. 
##### Vincent A. Voelz and Guangfeng Zhou. Journal of Computational Chemistry 35(30):2215–2224 (2014). 
#### DOI: [10.1002/jcc.23738](https://onlinelibrary-wiley-com.libproxy.temple.edu/doi/pdf/10.1002/jcc.23738)
#### In this example, we determine solution-state conformational populations of the 14-membered macrocycle cineromycin B, using a combination of previously published sparse Nuclear Magnetic Resonance (NMR) observables and replica exchange molecular dynamic/Quantum mehcanics (QM)-refined conformational ensembles. Cineromycin B is a 14-membered macrolide antibiotic that has gained attention for having activity against methicillin-resistant Staphylococcus aureus (MRSA). In this approach, we are able to calculate the consistency of computational modeling with experiment, and the relative importance of refence potentials and other model parameters.
-->

<h1 style="align: center;font-size: 18pt;"> Cineromycin B</h1>
<hr style="height:2.5px">
<h2 style="align: center;font-size: 12pt;"> Summary: In our previous work (DOI: <a href="https://onlinelibrary-wiley-com.libproxy.temple.edu/doi/pdf/10.1002/jcc.23738">10.1002/jcc.23738</a>), we determine solution-state conformational populations of the 14-membered macrocycle cineromycin B, using a combination of previously published sparse Nuclear Magnetic Resonance (NMR) observables and replica exchange molecular dynamic/Quantum mehcanics (QM)-refined conformational ensembles. Cineromycin B is a 14-membered macrolide antibiotic that has become increasingly known for having activity against methicillin-resistant Staphylococcus Aureus (MRSA). In this example, we show how to calculate the consistency of computational modeling with experiment, and the relative importance of reference potentials and other model parameters. </h2>
<hr style="height:2.5px">

In [1]:
import sys, os, glob, cPickle
import numpy as np
import biceps

BICePs - Bayesian Inference of Conformational Populations, Version 2.0


In [2]:
#########################################
# Lets' create input files for BICePs
############ Preparation ################
# Specify necessary argument values
path='NOE/*txt'                # precomputed distances for each state
states=100                     # number of states
indices='atom_indice_noe.txt'  # atom indices of each distance
exp_data='noe_distance.txt'    # experimental NOE data 
top='cineromycinB_pdbs/0.fixed.pdb'    # topology file 
dataFiles = 'noe_J'            # output directory of BICePs formated input file from this scripts
out_dir=dataFiles               

#p=Preparation('noe',states=states,indices=indices,
#    exp_data=exp_data,top=top,data_dir=data_dir)  # the type of data needs to be specified {'noe', 'J', 'cs_H', etc}
#p.write(out_dir=out_dir)      # raw data will be converted to a BICePs readable format to the folder specified

In [3]:
#########################################
# Let's create our ensemble of structures
############ Initialization #############    
# Specify necessary argument values
data = biceps.sort_data(dataFiles)   # sorting data in the folder and figure out what types of data are used
print data
print len(data),len(data[0])

energies_filename =  'cineromycinB_QMenergies.dat'
energies = np.loadtxt(energies_filename)*627.509  # convert from hartrees to kcal/mol
energies = energies/0.5959   # convert to reduced free energies F = f/kT
energies -= energies.min()  # set ground state to zero, just in case
outdir = 'results_ref_normal_new'
# Temporarily placing the number of steps here...
nsteps = 1000000 # number of steps of MCMC simulation

# Make a new directory if we have to
if not os.path.exists(outdir):
    os.mkdir(outdir)

[['noe_J/0.J', 'noe_J/0.noe'], ['noe_J/1.J', 'noe_J/1.noe'], ['noe_J/2.J', 'noe_J/2.noe'], ['noe_J/3.J', 'noe_J/3.noe'], ['noe_J/4.J', 'noe_J/4.noe'], ['noe_J/5.J', 'noe_J/5.noe'], ['noe_J/6.J', 'noe_J/6.noe'], ['noe_J/7.J', 'noe_J/7.noe'], ['noe_J/8.J', 'noe_J/8.noe'], ['noe_J/9.J', 'noe_J/9.noe'], ['noe_J/10.J', 'noe_J/10.noe'], ['noe_J/11.J', 'noe_J/11.noe'], ['noe_J/12.J', 'noe_J/12.noe'], ['noe_J/13.J', 'noe_J/13.noe'], ['noe_J/14.J', 'noe_J/14.noe'], ['noe_J/15.J', 'noe_J/15.noe'], ['noe_J/16.J', 'noe_J/16.noe'], ['noe_J/17.J', 'noe_J/17.noe'], ['noe_J/18.J', 'noe_J/18.noe'], ['noe_J/19.J', 'noe_J/19.noe'], ['noe_J/20.J', 'noe_J/20.noe'], ['noe_J/21.J', 'noe_J/21.noe'], ['noe_J/22.J', 'noe_J/22.noe'], ['noe_J/23.J', 'noe_J/23.noe'], ['noe_J/24.J', 'noe_J/24.noe'], ['noe_J/25.J', 'noe_J/25.noe'], ['noe_J/26.J', 'noe_J/26.noe'], ['noe_J/27.J', 'noe_J/27.noe'], ['noe_J/28.J', 'noe_J/28.noe'], ['noe_J/29.J', 'noe_J/29.noe'], ['noe_J/30.J', 'noe_J/30.noe'], ['noe_J/31.J', 'noe_J/31.no

In [ ]:
######################
# Main:
######################

lambda_values = [0.0,0.5,1.0]
res = biceps.list_res(data)
print res
ref=['uniform','exp']
uncern=[[0.05,20.0,1.02],[0.05,5.0,1.02]]
gamma = [0.2,5.0,1.01]

for j in lambda_values:
    verbose = False # True
    lam = j
    # We will instantiate a number of Structure() objects to construct the ensemble
    ensemble = []
    for i in range(energies.shape[0]):
        print '\n#### STRUCTURE %d ####'%i
        ensemble.append([])
        for k in range(len(data[0])):
            File = data[i][k]
            if verbose:
                print File
            R = biceps.init_res('cineromycinB_pdbs/0.fixed.pdb',
                              lam,energies[i],File,ref[k],uncern[k],gamma)
            ensemble[-1].append(R)
    print ensemble

    ##########################################
    # Next, let's do some posterior sampling
    ########## Posterior Sampling ############
    
    sampler = biceps.PosteriorSampler(ensemble)
    sampler.compile_nuisance_parameters()

    sampler.sample(nsteps)  # number of steps

    print 'Processing trajectory...',
    print 'Writing results...',
    sampler.traj.process_results(os.path.join(outdir,'traj_lambda%2.2f.npz'%lam))  # compute averages, etc.
    print '...Done.'
    
    sampler.traj.read_results(os.path.join(outdir,'traj_lambda%2.2f.npz'%lam))

    print 'Pickling the sampler object ...',
    outfilename = 'sampler_lambda%2.2f.pkl'%lam
    print outfilename,
    fout = open(os.path.join(outdir, outfilename), 'wb')
    # Pickle dictionary using protocol 0.
    cPickle.dump(sampler, fout)
    fout.close()
    print '...Done.'

['J', 'noe']

#### STRUCTURE 0 ####

#### STRUCTURE 1 ####

#### STRUCTURE 2 ####

#### STRUCTURE 3 ####

#### STRUCTURE 4 ####

#### STRUCTURE 5 ####

#### STRUCTURE 6 ####

#### STRUCTURE 7 ####

#### STRUCTURE 8 ####

#### STRUCTURE 9 ####

#### STRUCTURE 10 ####

#### STRUCTURE 11 ####

#### STRUCTURE 12 ####

#### STRUCTURE 13 ####

#### STRUCTURE 14 ####

#### STRUCTURE 15 ####

#### STRUCTURE 16 ####

#### STRUCTURE 17 ####

#### STRUCTURE 18 ####

#### STRUCTURE 19 ####

#### STRUCTURE 20 ####

#### STRUCTURE 21 ####

#### STRUCTURE 22 ####

#### STRUCTURE 23 ####

#### STRUCTURE 24 ####

#### STRUCTURE 25 ####

#### STRUCTURE 26 ####

#### STRUCTURE 27 ####

#### STRUCTURE 28 ####

#### STRUCTURE 29 ####

#### STRUCTURE 30 ####

#### STRUCTURE 31 ####

#### STRUCTURE 32 ####

#### STRUCTURE 33 ####

#### STRUCTURE 34 ####

#### STRUCTURE 35 ####

#### STRUCTURE 36 ####

#### STRUCTURE 37 ####

#### STRUCTURE 38 ####

#### STRUCTURE 39 ####

#### STRUCTURE 40 ####

#### STRUCTU


Accepted 78.5819 % 


Accepted [ 24.5881  24.2902  23.8021   5.9015] % 

Processing trajectory... Writing results... ...Done.
Pickling the sampler object ... sampler_lambda0.00.pkl ...Done.

#### STRUCTURE 0 ####

#### STRUCTURE 1 ####

#### STRUCTURE 2 ####

#### STRUCTURE 3 ####

#### STRUCTURE 4 ####

#### STRUCTURE 5 ####

#### STRUCTURE 6 ####

#### STRUCTURE 7 ####

#### STRUCTURE 8 ####

#### STRUCTURE 9 ####

#### STRUCTURE 10 ####

#### STRUCTURE 11 ####

#### STRUCTURE 12 ####

#### STRUCTURE 13 ####

#### STRUCTURE 14 ####

#### STRUCTURE 15 ####

#### STRUCTURE 16 ####

#### STRUCTURE 17 ####

#### STRUCTURE 18 ####

#### STRUCTURE 19 ####

#### STRUCTURE 20 ####

#### STRUCTURE 21 ####

#### STRUCTURE 22 ####

#### STRUCTURE 23 ####

#### STRUCTURE 24 ####

#### STRUCTURE 25 ####

#### STRUCTURE 26 ####

#### STRUCTURE 27 ####

#### STRUCTURE 28 ####

#### STRUCTURE 29 ####

#### STRUCTURE 30 ####

#### STRUCTURE 31 ####

#### STRUCTURE 32 ####

#### STRUCTURE 33 ####

##


Accepted 75.1902 % 


Accepted [ 24.5618  24.2425  23.8593   2.5266] % 

Processing trajectory... Writing results... ...Done.
Pickling the sampler object ... sampler_lambda0.50.pkl ...Done.

#### STRUCTURE 0 ####

#### STRUCTURE 1 ####

#### STRUCTURE 2 ####

#### STRUCTURE 3 ####

#### STRUCTURE 4 ####

#### STRUCTURE 5 ####

#### STRUCTURE 6 ####

#### STRUCTURE 7 ####

#### STRUCTURE 8 ####

#### STRUCTURE 9 ####

#### STRUCTURE 10 ####

#### STRUCTURE 11 ####

#### STRUCTURE 12 ####

#### STRUCTURE 13 ####

#### STRUCTURE 14 ####

#### STRUCTURE 15 ####

#### STRUCTURE 16 ####

#### STRUCTURE 17 ####

#### STRUCTURE 18 ####

#### STRUCTURE 19 ####

#### STRUCTURE 20 ####

#### STRUCTURE 21 ####

#### STRUCTURE 22 ####

#### STRUCTURE 23 ####

#### STRUCTURE 24 ####

#### STRUCTURE 25 ####

#### STRUCTURE 26 ####

#### STRUCTURE 27 ####

#### STRUCTURE 28 ####

#### STRUCTURE 29 ####

#### STRUCTURE 30 ####

#### STRUCTURE 31 ####

#### STRUCTURE 32 ####

#### STRUCTURE 33 ####

##

In [ ]:
#########################################
# Let's do analysis using MBAR and plot figures
############ MBAR and Figures ###########
# Specify necessary argument values
%matplotlib inline
A = biceps.Analysis(states, outdir) 
A.plot()

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../../theme.css", "r").read()
    return HTML(styles)
css_styling()